## Prerequisites

Ensure Vertex AI API is enabled in GCP:

![Vertex AI API](img/vertex_ai.png)

Authenticate with gcloud CLI: 

https://cloud.google.com/docs/authentication/provide-credentials-adc#google-idp

## RAG with Gemini Pro 1.5

### Install & import dependencies

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma langchain-google-vertexai langsmith

In [ ]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.document_loaders import TextLoader
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_vertexai import ChatVertexAI

### Enable LangSmith tracing (optional)

In [ ]:
# LangSmith tracing
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Demo1"
# os.environ["LANGCHAIN_API_KEY"] = "<Your LangSmith API key>"

### Indexing

You can skip this section if the data is already indexed.
If the `chroma_db` folder is present - it is.

In [ ]:
# Load markdown documents from the specified folder
loader = DirectoryLoader('./gitlab_engineering/infrastructure', glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
docs = loader.load()

# Split the documents
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, return_each_line=False, strip_headers=False)

splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]

In [ ]:
# Create an instance of embeddings model integration
embedding = VertexAIEmbeddings(model_name='textembedding-gecko@003')

In [ ]:
# Vectorize and index the split documents
persist_directory="./chroma_db"
Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=persist_directory)

### Retrieval & generation

In [ ]:
# Create a retriever for the indexed data
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = vectorstore.as_retriever(search_kwargs={'k': 8})

# Pull RAG prompt template. See: https://smith.langchain.com/hub/rlm/rag-prompt
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create an instance of a chat model integration
llm = ChatVertexAI(model_name='gemini-1.5-pro-preview-0409')

# Create a question answering RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Invoke the chain
rag_chain.invoke("What services are not covered by canary stage?")